# Off menu transcripts extraction 

This notebook serves as a development environment for the logic to scrape and save episode transcripts html. The final production code is located in off_menu/data_extraction.py.

In [ ]:
# Ensure imports can find my utils:

import sys
import os

notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..', '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root added to sys.path: {project_root}")
print(f"Current sys.path: {sys.path}")

# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Project specific imports
from off_menu.utils import extract_html
from off_menu.utils import save_text_to_file
from off_menu.config import episodes_list_url

Project root added to sys.path: c:\Users\jbara\Data science projects (store here not desktop on onedrive)\Off Menu project
Current sys.path: ['c:\\Users\\jbara\\Data science projects (store here not desktop on onedrive)\\Off Menu project', 'C:\\Users\\jbara\\miniconda3\\python312.zip', 'C:\\Users\\jbara\\miniconda3\\DLLs', 'C:\\Users\\jbara\\miniconda3\\Lib', 'C:\\Users\\jbara\\miniconda3', 'c:\\Users\\jbara\\OneDrive\\Desktop\\Data_science\\Python projects\\Off Menu project\\.venv', '', 'c:\\Users\\jbara\\OneDrive\\Desktop\\Data_science\\Python projects\\Off Menu project\\.venv\\Lib\\site-packages', 'c:\\Users\\jbara\\OneDrive\\Desktop\\Data_science\\Python projects\\Off Menu project\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\jbara\\OneDrive\\Desktop\\Data_science\\Python projects\\Off Menu project\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jbara\\OneDrive\\Desktop\\Data_science\\Python projects\\Off Menu project\\.venv\\Lib\\site-packages\\Pythonwin']


## Configuration and filepaths

In [3]:
test_temp_dir = os.path.join(project_root, 'data/test_temp')

## Test access and parse an episode; Access and clean first sentence; Access all sentences in an episode

In [ ]:
#  Access and parse an episode
f = requests.get(
    "https://podscripts.co/podcasts/off-menu-with-ed-gamble-and-james-acaster/ep-284-meera-sodha"
)
ep_284 = BeautifulSoup(f.text, features="html.parser")

#  test - access and clean first sentence
first_sentence = ep_284.find("div", class_ = "single-sentence").text
#print(first_sentence.replace("  ", "").replace("\n", "").replace("  ", " "))

#  test - access all sentences in an episode
all_sentences = ep_284.find_all("div", class_ = "single-sentence")
print(all_sentences[0].text[:11])

NameError: name 'requests' is not defined

## Extract and save a transcript html

### Retrieve ep_and_mentions dataframe

In [ ]:
test_temp_dir = os.path.join(project_root, 'data/test_temp')
ep_and_mentions_filepath = os.path.join(test_temp_dir, 'ep_and_mentions.parquet')
try:
    ep_and_mentions_df = pd.read_parquet(ep_and_mentions_filepath)
except FileNotFoundError:
    print(f"Error: The file was not found at {ep_and_mentions_filepath}. Did it save correctly?")

# Test if it's loaded correctly
print("---ep_and_mentions_df head---")
print(ep_and_mentions_df.head())

   episode_number     guest_name  \
0               1  Scroobius Pip   
1               2     Grace Dent   
2               3  Richard Osman   
3               4     Nish Kumar   
4               5    Aisling Bea   

                                                 url  \
0  https://podscripts.co/podcasts/off-menu-with-e...   
1  https://podscripts.co/podcasts/off-menu-with-e...   
2  https://podscripts.co/podcasts/off-menu-with-e...   
3  https://podscripts.co/podcasts/off-menu-with-e...   
4  https://podscripts.co/podcasts/off-menu-with-e...   

                     restaurants_mentioned  
0                [Oli Baba’s, Kerb Camden]  
1                     [Little Owl, Trullo]  
2  [Five Guys, Cora Pearl, Berners Tavern]  
3       [Bademiya, The Owl & The Pussycat]  
4       [Café Gratitude, Burger & Lobster]  


### Function to access and save a transcript html based on url

In [ ]:
eps_to_process = ep_and_mentions_df.head(2)

def save_transcript_html(eps_dataframe, directory):
    for index, row in eps_dataframe.iterrows():
        episode_num = row['episode_number'] 
        episode_url = row['url']
        html_content_str = extract_html(episode_url).text
        save_text_to_file(html_content_str, f'ep_{episode_num}.html', directory)
    
save_transcript_html(eps_to_process, test_temp_dir)